# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [204]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [205]:
csv_path=os.path.join("output_data","cities.csv")
csv=pd.read_csv(csv_path)
csv_df=pd.DataFrame(csv)
csv_df


,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Kalmunai,7.42,81.82,78.40,85,36,8.57,LK,1611975828
1,1,Rikitea,-23.12,-134.97,80.46,70,38,9.26,PF,1611975828
2,2,Santa Maria,-29.68,-53.81,75.20,83,0,9.22,BR,1611975567
3,3,Butembo,0.15,29.28,59.54,69,12,2.66,CD,1611975829
4,4,Cidreira,-30.18,-50.21,75.16,89,39,4.92,BR,1611975829
...,...,...,...,...,...,...,...,...,...,...
543,543,Udimskiy,61.14,45.91,17.91,97,100,6.04,RU,1611976037
544,544,Itacurubí del Rosario,-24.50,-56.68,73.85,96,53,4.27,PY,1611976037
545,545,Charters Towers,-20.10,146.27,88.97,31,76,9.89,AU,1611976020
546,546,Três Lagoas,-20.75,-51.68,71.87,92,84,4.29,BR,1611976038


In [206]:
#marker for setting humidity to the max value for the heatmap intencity
csv_df['Humidity'].max()

100

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [207]:
lat_lng=csv_df[['Lat','Lng']]
hum=csv_df['Humidity']


In [208]:
fig=gmaps.figure(center=(50.0, -35.0), zoom_level=2
)
heat_layer=gmaps.heatmap_layer(lat_lng, weights=hum,
                           dissipating=False,max_intensity=100,
                           point_radius=2)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [209]:
hotel_df=csv_df.dropna()
hotel_df=pd.DataFrame(hotel_df)
hotel_df


,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Kalmunai,7.42,81.82,78.40,85,36,8.57,LK,1611975828
1,1,Rikitea,-23.12,-134.97,80.46,70,38,9.26,PF,1611975828
2,2,Santa Maria,-29.68,-53.81,75.20,83,0,9.22,BR,1611975567
3,3,Butembo,0.15,29.28,59.54,69,12,2.66,CD,1611975829
4,4,Cidreira,-30.18,-50.21,75.16,89,39,4.92,BR,1611975829
...,...,...,...,...,...,...,...,...,...,...
543,543,Udimskiy,61.14,45.91,17.91,97,100,6.04,RU,1611976037
544,544,Itacurubí del Rosario,-24.50,-56.68,73.85,96,53,4.27,PY,1611976037
545,545,Charters Towers,-20.10,146.27,88.97,31,76,9.89,AU,1611976020
546,546,Três Lagoas,-20.75,-51.68,71.87,92,84,4.29,BR,1611976038


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [210]:
hotel_df['Hotel Name']=''
sm_hotel_df=hotel_df
sm_hotel_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,0,Kalmunai,7.42,81.82,78.40,85,36,8.57,LK,1611975828,
1,1,Rikitea,-23.12,-134.97,80.46,70,38,9.26,PF,1611975828,
2,2,Santa Maria,-29.68,-53.81,75.20,83,0,9.22,BR,1611975567,
3,3,Butembo,0.15,29.28,59.54,69,12,2.66,CD,1611975829,
4,4,Cidreira,-30.18,-50.21,75.16,89,39,4.92,BR,1611975829,
...,...,...,...,...,...,...,...,...,...,...,...
543,543,Udimskiy,61.14,45.91,17.91,97,100,6.04,RU,1611976037,
544,544,Itacurubí del Rosario,-24.50,-56.68,73.85,96,53,4.27,PY,1611976037,
545,545,Charters Towers,-20.10,146.27,88.97,31,76,9.89,AU,1611976020,
546,546,Três Lagoas,-20.75,-51.68,71.87,92,84,4.29,BR,1611976038,


In [211]:
sm_hotel_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,0,Kalmunai,7.42,81.82,78.40,85,36,8.57,LK,1611975828,
1,1,Rikitea,-23.12,-134.97,80.46,70,38,9.26,PF,1611975828,
2,2,Santa Maria,-29.68,-53.81,75.20,83,0,9.22,BR,1611975567,
3,3,Butembo,0.15,29.28,59.54,69,12,2.66,CD,1611975829,
4,4,Cidreira,-30.18,-50.21,75.16,89,39,4.92,BR,1611975829,
...,...,...,...,...,...,...,...,...,...,...,...
543,543,Udimskiy,61.14,45.91,17.91,97,100,6.04,RU,1611976037,
544,544,Itacurubí del Rosario,-24.50,-56.68,73.85,96,53,4.27,PY,1611976037,
545,545,Charters Towers,-20.10,146.27,88.97,31,76,9.89,AU,1611976020,
546,546,Três Lagoas,-20.75,-51.68,71.87,92,84,4.29,BR,1611976038,


In [212]:
#filter the df to get ideal conditions for a vacation
sm_hotel_df = hotel_df[(hotel_df["Wind Speed"] <= 10) & (hotel_df["Cloudiness"] == 0) & \
                                   (hotel_df["Max Temp"] >= 65) & (hotel_df["Max Temp"] <= 90)].dropna()
len(sm_hotel_df)

21

In [214]:
ser_hotel_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {
    "radius": "5000",
    "type": "lodging",
    "key": g_key,
}
count = 0
for index, row in sm_hotel_df.iterrows(): #add this after hotel_df for looping in range [0:28]

    # get city name from df
    city_name = row['City']

#     # add location to params dict
  
    lat_l=row['Lat']
    lng_l=row["Lng"]
    params['location'] = f'{lat_l},{lng_l}'
#  
    try:
        response = requests.get(ser_hotel_url, params=params).json()

        results = response['results']
#         print(f"Closest Hotel in {city_name} is {results[0]['name']}.")
        
        sm_hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        count+=1
    except (KeyError, IndexError):
#         print("Missing field/result... skipping.")
    
        
        
        print("------------")
    

------------
------------


In [215]:
sm_hotel_df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
2,2,Santa Maria,-29.68,-53.81,75.20,83,0,9.22,BR,1611975567,Hotel Morotin
10,10,Busselton,-33.65,115.33,82.00,45,0,4.00,AU,1611975627,Observatory Guest House
15,15,Mataura,-46.19,168.86,70.48,36,0,0.94,NZ,1611975833,Ellie's Villa
22,22,Saint-Philippe,-21.36,55.77,78.80,57,0,6.91,RE,1611975836,"Chambres d'hôte ""La Trinité"""
39,39,Saquarema,-22.92,-42.51,82.40,83,0,9.22,BR,1611975841,Hotel Fazenda Sol e Lua


In [216]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
sm_hotel_info = [info_box_template.format(**row) for index, row in sm_hotel_df.iterrows()]
locations = sm_hotel_df[["Lat", "Lng"]]


In [217]:
sm_hotel_name=sm_hotel_df['Hotel Name'].tolist()
print(sm_hotel_name)

['Hotel Morotin', 'Observatory Guest House', "Ellie's Villa", 'Chambres d\'hôte "La Trinité"', 'Hotel Fazenda Sol e Lua', "Ron and Tony's Bed & Breakfast", 'Hostal Morales', '', 'Four Points by Sheraton Caguas Real Hotel & Casino', 'JAGDISH STAINLESS STEEL WORK SHOP', 'Pousada dos Navegantes', 'Huadu Hotel', 'True Treasure Hotel', 'hotêl doufoulougou', 'Bom Será - Marcinho e Fernanda', 'Screensaver', 'Al Mumtaz Transport Jiwani', 'Auberge la Victoire', 'Hotel San Isidro Oficial', '"Plantation Bed and Breakfast', '']


In [218]:
# Add marker layer ontop of heat map
list_ll=hotel_df[['Lat','Lng']]
hum2=hotel_df['Humidity']
marker_locations=sm_hotel_df[['Lat','Lng']]
info_box_content=[f"Hotel Name: {name}" for name in sm_hotel_info]
fig = gmaps.figure(center=(50.0, -35.0), zoom_level=2
)
markers = gmaps.marker_layer(marker_locations,
    info_box_content=[f"Hotel Name: {name}" for name in sm_hotel_info])

fig.add_layer(markers)
heat_layer=gmaps.heatmap_layer(list_ll, weights=hum2,
                           dissipating=False,max_intensity=100,
                           point_radius=1,)
fig.add_layer(heat_layer)
fig
    

# Display figure


Figure(layout=FigureLayout(height='420px'))